# 01. Game Walkthrough

This notebook demonstrates how to interact with the Euchre game engine and run a simulation.

In [ ]:
import sys
import os

# Add parent directory to path for imports
sys.path.insert(0, os.path.abspath(os.path.join('..')))

from euchre.engine.state import EuchreGameState, GamePhase
from euchre.agents import RandomAgent

print('Imports successful!')

### 1. Initialize the Game

Create the state and start a hand.

In [ ]:
game = EuchreGameState()
game.start_hand()

print(f'Dealer: Player {game.dealer_index}')
print(f'Up Card: {game.up_card}')
print(f'Current Phase: {game.phase.name}')

### 2. Create Agents

We'll create 4 Random Agents to play.

In [ ]:
agents = [RandomAgent(f'Bot{i}') for i in range(4)]
print(f'Created {len(agents)} agents')

### 3. Run the Game Loop

We'll loop through the bidding and playing phases.

In [ ]:
step_count = 0
max_steps = 100  # Safety limit

while game.phase not in [GamePhase.HAND_OVER, GamePhase.GAME_OVER] and step_count < max_steps:
    current_player = game.current_player_index
    agent = agents[current_player]
    
    print(f'\n--- Step {step_count}: P{current_player} ({game.phase.name}) ---')
    
    if game.phase == GamePhase.BIDDING_ROUND_1:
        # Agent decides whether to order up
        decision = agent.pick_up_card(game)
        if decision:
            game.order_up(current_player)
        else:
            game.pass_turn()
            
    elif game.phase == GamePhase.BIDDING_ROUND_2:
        # Agent calls a suit or passes
        suit = agent.call_suit(game)
        if suit:
            game.call_suit(current_player, suit)
        else:
            game.pass_turn()
            
    elif game.phase == GamePhase.PLAYING:
        # Agent plays a card
        card = agent.play_card(game)
        # Find the card index in the hand
        hand = game.hands[current_player]
        try:
            card_idx = next(i for i, c in enumerate(hand) if c.rank == card.rank and c.suit == card.suit)
            game.play_card(current_player, card_idx)
        except StopIteration:
            print(f'ERROR: Card {card} not found in hand {hand}')
            break
    
    step_count += 1

print(f'\n=== Game Complete ===')
print(f'Final Phase: {game.phase.name}')
print(f'Team 0 Score: {game.team_scores[0]}')
print(f'Team 1 Score: {game.team_scores[1]}')

### 4. Inspect the Final State

In [ ]:
print(f'Trump Suit: {game.trump_suit}')
print(f'Maker Team: {game.maker_team}')
print(f'Tricks Taken: Team 0 = {game.tricks_taken[0]}, Team 1 = {game.tricks_taken[1]}')